In [1]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Cargar los datos
data = pd.read_csv('../../data/partidos_limpio.csv')

# Extracción de goles de local y visitante de la columna 'Score'
data['Home Goals'] = data['Score'].apply(lambda x: int(x.split('-')[0].strip()))
data['Away Goals'] = data['Score'].apply(lambda x: int(x.split('-')[1].strip()))

# Creación de una columna para el resultado del partido
# 1 si gana el local, -1 si gana el visitante, 0 si empate
data['Match Result'] = data.apply(lambda x: 1 if x['Home Goals'] > x['Away Goals'] else (-1 if x['Home Goals'] < x['Away Goals'] else 0), axis=1)

# Codificar variables categóricas
encoder = LabelEncoder()
data['Season_encoded'] = encoder.fit_transform(data['Season'])
data['Round_encoded'] = encoder.fit_transform(data['Round'])
data['Home_encoded'] = encoder.fit_transform(data['Home'])
data['Away_encoded'] = encoder.fit_transform(data['Away'])
data['Venue_encoded'] = encoder.fit_transform(data['Venue'])
data['Referee_encoded'] = encoder.fit_transform(data['Referee'])

# Crear y escalar características numéricas
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data[['Home Goals', 'Away Goals', 'Season_encoded', 'Round_encoded', 'Home_encoded', 'Away_encoded', 'Venue_encoded', 'Referee_encoded']])

# Preparar los datos para el entrenamiento
X = scaled_features
y = data['Match Result'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construir el modelo de red neuronal
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')  # Utilizamos activación lineal para una clasificación simple
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2, verbose=1)  # Cambio a verbose=1 para mostrar el progreso de entrenamiento

# Evaluación del modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

test_loss, test_accuracy


c:\Users\albab\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (





Epoch 1/50


37/37 [==============================] - 1s 6ms/step - loss: 0.4896 - accuracy: 0.3515 - val_loss: 0.2716 - val_accuracy: 0.5109
Epoch 2/50
37/37 [==============================] - 0s 2ms/step - loss: 0.1916 - accuracy: 0.5940 - val_loss: 0.1903 - val_accuracy: 0.6304
Epoch 3/50
37/37 [==============================] - 0s 2ms/step - loss: 0.1291 - accuracy: 0.6594 - val_loss: 0.1624 - val_accuracy: 0.6087
Epoch 4/50
37/37 [==============================] - 0s 2ms/step - loss: 0.1036 - accuracy: 0.6621 - val_loss: 0.1369 - val_accuracy: 0.6739
Epoch 5/50
37/37 [==============================] - 0s 2ms/step - loss: 0.0803 - accuracy: 0.6921 - val_loss: 0.1214 - val_accuracy: 0.6848
Epoch 6/50
37/37 [==============================] - 0s 2ms/step - loss: 0.0652 - accuracy: 0.7003 - val_loss: 0.1133 - val_accuracy: 0.6848
Epoch 7/50
37/37 [==============================] - 0s 3ms/step - loss: 0.0535 - accuracy: 0.7057 - val_loss: 0.1016 - val_accuracy: 0.7065
Epoch 8/50
37/3

(0.03126104548573494, 0.643478274345398)

C:\Users\albab\AppData\Local\Temp\ipykernel_31012\2693956746.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_upcoming['Referee'] = 'Unknown'


NameError: name 'preprocessor' is not defined

In [6]:
updated_data_path = '../../data/partidos_2023-2024_limpio.csv'
updated_data = pd.read_csv(updated_data_path)

# Filtrar partidos sin resultado en el score
upcoming_matches = updated_data[updated_data['Score'].isna()]


# Preparar los datos de los próximos partidos para la predicción
features_upcoming = upcoming_matches[['Home', 'Away']]

# Añadir columna 'Referee' con un valor predeterminado si no está presente
if 'Referee' not in features_upcoming.columns:
    features_upcoming['Referee'] = 'Unknown'


# Codificar variables categóricas utilizando las etiquetas ya ajustadas
features_upcoming['Home_encoded'] = encoder.transform(features_upcoming['Home'])
features_upcoming['Away_encoded'] = encoder.transform(features_upcoming['Away'])

# Escalar características numéricas
scaled_features_upcoming = scaler.transform(features_upcoming[['Home_encoded', 'Away_encoded']])

# Hacer las predicciones con el modelo entrenado
predicted_results = model.predict(scaled_features_upcoming)

# Interpretar las predicciones
predicted_results = ['Home Wins' if result > 0 else 'Away Wins' if result < 0 else 'Draw' for result in predicted_results]

# Agregar las predicciones al dataframe de próximos partidos
upcoming_matches['Predicted Result'] = predicted_results

# Mostrar los resultados
print(upcoming_matches[['Home', 'Away', 'Predicted Result']])



C:\Users\albab\AppData\Local\Temp\ipykernel_31012\224527514.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_upcoming['Referee'] = 'Unknown'


ValueError: y contains previously unseen labels: 'Bayern Munich'

In [3]:
# Importar las librerías necesarias
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Cargar los datos
data = pd.read_csv('../../data/partidos_2023-2024_limpio.csv')

# Filtrar partidos sin resultado en el score
upcoming_matches = data[data['Score'].isna()]

# Preparar los datos de los próximos partidos para la predicción
features_upcoming = upcoming_matches[['Home', 'Away']]

# Codificar variables categóricas utilizando las etiquetas ya ajustadas
features_upcoming['Home_encoded'] = encoder.transform(features_upcoming['Home'])
features_upcoming['Away_encoded'] = encoder.transform(features_upcoming['Away'])

# Escalar características numéricas
scaled_features_upcoming = scaler.transform(features_upcoming[['Home_encoded', 'Away_encoded']])

# Hacer las predicciones con el modelo entrenado
predicted_results = model.predict(scaled_features_upcoming)

# Interpretar las predicciones
predicted_results = ['Home Wins' if result > 0 else 'Away Wins' if result < 0 else 'Draw' for result in predicted_results]

# Agregar las predicciones al dataframe de próximos partidos
upcoming_matches['Predicted Result'] = predicted_results

# Mostrar los resultados
print(upcoming_matches[['Home', 'Away', 'Predicted Result']])


ValueError: y contains previously unseen labels: 'Bayern Munich'